In [1]:
import torch
from tqdm.notebook import tqdm

In [2]:
def shift(c, size, i):
    res = torch.zeros(size, device=c.device)
    res[:i] = c[i:]
    return res

def toeplitz(c, r):
    if len(c.size()) == 3:
        IC, OC, l_r = r.size()
        l_c = c.size()[-1]
        res = torch.zeros(IC,OC, l_c, l_r, device=c.device)
        # p = torch.zeros(len(c))
        # c = torch.cat([p,c.flip(0)])
        c = c.flip(2)
        c = torch.cat([c,c],dim=2)
        for i in range(1, l_c + 1):
            if i <= l_r:
                res[:,:,i-1,:i] = c[:,:,-i:]
            else:
                res[:,:,i-1,:] = c[:,:,-i:-i+l_r]
        return res
    if len(c.size()) == 1:
        l_r = r.size()[0]
        l_c = c.size()[0]
        res = torch.zeros(l_c, l_r, device=c.device)
        # p = torch.zeros(len(c))
        # c = torch.cat([p,c.flip(0)])
        c = c.flip(0)
        c = torch.cat([c,c])
        for i in range(1, l_c + 1):
            if i <= l_r:
                res[i-1,:i] = c[-i:]
            else:
                res[i-1,:] = c[-i:-i+l_r]
        return res

def vector_to_matrix(input, output_shape):
    B, C, output_h, output_w = output_shape
    output = torch.zeros(output_shape, dtype=input.dtype, device=input.device)
    for i in range(output_h):
        st = i*output_w
        nd = st + output_w
        output[:,:, i, :] = input[:,:,st:nd]
    # flip the output matrix up-down to get correct result
    output=torch.flip(output,dims=[-2])
    return output

def matrix_to_vector(input):
    B, C, input_h, input_w = input.shape
    output_vector = torch.zeros(B, C, input_h*input_w, dtype=input.dtype, device=input.device)
    # flip the input matrix up-down because last row should go first
    input = torch.flip(input,dims=[-2]) 
    for i in range(input.size()[2]):
        st = i*input_w
        nd = st + input_w
        output_vector[:,:,st:nd] = input[:,:,i]
        
    return output_vector


In [3]:
def lol(I,F):
    st = time.time()
    I_B, I_C, I_row_num, I_col_num = I.size() 

    # number of columns and rows of the filter
    F_CO, F_CI, F_row_num, F_col_num = F.size()

    #  calculate the output dimensions
    output_row_num = I_row_num + F_row_num - 1
    output_col_num = I_col_num + F_col_num - 1

    F_zero_padded = torch.zeros(F_CO, F_CI, output_row_num, output_col_num, device=I.device)
    F_zero_padded[:, :, -F_row_num:, :F_col_num] = F

    toeplitz_list = []
    for i in range(F_zero_padded.shape[2]-1, -1, -1): # iterate from last row to the first row
        c = F_zero_padded[:, :, i, :] # i th row of the F 
        r = torch.zeros(F_CO, F_CI, I_col_num, device=I.device)
        r[:,:,0] = c[:,:,0]
        toeplitz_m = toeplitz(c, r)
        toeplitz_list.append(toeplitz_m)

    c = torch.Tensor(range(1, F_zero_padded.shape[2]+1)).to(I.device)
    r = torch.zeros(I_row_num, device=I.device)
    r[0] = c[0]
    doubly_indices = toeplitz(c, r).to(torch.int)
    # print(doubly_indices)

    toeplitz_shape = toeplitz_list[0].shape # shape of one toeplitz matrix
    h = toeplitz_shape[2]*doubly_indices.shape[0]
    w = toeplitz_shape[3]*doubly_indices.shape[1]
    doubly_blocked_shape = [F_CO, F_CI, h, w]
    doubly_blocked = torch.zeros(doubly_blocked_shape, device=I.device)

    # tile toeplitz matrices for each row in the doubly blocked matrix
    IC, OC, b_h, b_w = toeplitz_shape # hight and withs of each block
    for i in range(doubly_indices.shape[0]):
        for j in range(doubly_indices.shape[1]):
            start_i = i * b_h
            start_j = j * b_w
            end_i = start_i + b_h
            end_j = start_j + b_w
            doubly_blocked[:,:,start_i: end_i, start_j:end_j] = toeplitz_list[doubly_indices[i,j]-1]
    
    print("sad")
    # print(doubly_blocked.shape)
    vectorized_I = matrix_to_vector(I)
    # print(vectorized_I.shape)
    # print(vectorized_I)
    output_vect = torch.zeros(I_B, F_CO, output_row_num * output_col_num, device=I.device)
    # print(doubly_blocked.shape)
    for i in tqdm(range(I_B)):
        # for j in range(F_CO):
            for k in range(I_C):
                output_vect[i,:,:] += torch.matmul(doubly_blocked[:,k].view(F_CO,h,w), vectorized_I[i,k,:].view(I_row_num * I_col_num))
    out_shape = [I_B, F_CO, output_row_num, output_col_num]
    my_output = vector_to_matrix(output_vect, out_shape)
    print(time.time() - st)
    return my_output

In [9]:
import time
# device = torch.device("cuda:0")
device = torch.device("cpu")
I = torch.randn(32,16,16,16).to(device)
F = torch.randn(64,16,3,3).to(device)
# I = torch.Tensor([[[[1, 2, 3], [4, 5, 6], [7,8,9]], [[1, 2, 3], [4, 5, 6], [7,8,9]]]])
# I = torch.ones(1,2,3,3)
model = torch.nn.Conv2d(16,64,3,padding=2).to(device)
# F = torch.Tensor([[[[10, 20, 30], [40, 50, 60], [70, 80, 90]],[[10, 20, 30], [40, 50, 60], [70, 80, 90]]]])
# F = torch.ones(1,2,3,3)
model.weight.data = F.flip(dims=[-1,-2])
model.bias.data = torch.zeros_like(model.bias)
# print(I.shape)
m_res = model(I)
st = time.time()
a = lol(I,F)#.numpy()
# print(torch.prod(torch.Tensor(a.shape)))
print(time.time() - st)
print(((a - m_res).abs() < 1e-4).sum() / torch.prod(torch.Tensor(list(a.shape))))

sad



3.24786376953125
3.2758641242980957
tensor(1.)
